In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

import warnings

# Установка параметров для игнорирования предупреждений
warnings.filterwarnings('ignore')

pd.set_option('display.width', 200)


In [2]:
# Загрузка данных
data = pd.read_csv('data\\train.csv')

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Делим на тестовую и тренировочную выборки
# Каждую выборку будем трансформировать по одному ыалгоритму

# Признаки (X) и целевая переменная (y)
X = data.drop(['Survived'], axis=1)
y = data['Survived']

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Находим самое частое значение в столбце 'Embarked'
most_common_embarked = X_train['Embarked'].mode()[0]

# Заменяем пропущенные значения в столбце 'Embarked' на самое частое значение
X_train['Embarked'].fillna(most_common_embarked, inplace=True)

In [6]:
# Сгруппировать данные по кабине и вычислить среднее значение цены билета
cabin_means = X_train.groupby('Cabin')['Fare'].mean()

# Заменить пропущенные значения кабины на ближайшее среднее значение цены билета
for index, row in X_train.iterrows():
    if pd.isnull(row['Cabin']):
        closest_cabin = cabin_means.idxmin()  # Найти ближайшее среднее значение кабины
        X_train.at[index, 'Cabin'] = closest_cabin  # Заменить пропущенное значение кабины

In [7]:
# Получим список числовых столбцов
numerical_columns = X_train.select_dtypes(include=['int', 'float']).columns.tolist()

# Создадим копию X_train для обработки пропущенных значений
X_train_processed = X_train.copy()

# Заполним пропущенные значения в числовых столбцах медианным значением
imputer = SimpleImputer(strategy='median')
X_train[numerical_columns] = imputer.fit_transform(X_train[numerical_columns])

# Удалим строки с пропущенными значениями в столбце "Age" для более точной кластеризации
X_train_without_missing_age = X_train.dropna(subset=['Age'])

# Количество кластеров
n_clusters = 6

# Кластеризуем данные по возрасту
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
X_train_without_missing_age['AgeCluster'] = kmeans.fit_predict(X_train_without_missing_age[['Age']])

# Найдем медиану каждого кластера
age_cluster_medians = X_train_without_missing_age.groupby('AgeCluster')['Age'].median()

# Заменим пропущенные значения возраста медианным значением соответствующего кластера
for index, row in X_train[X_train['Age'].isnull()].iterrows():
    age_cluster = kmeans.predict([[row['Age']]])[0]
    median_age = age_cluster_medians[age_cluster]
    X_train.at[index, 'Age'] = median_age

In [8]:
# Преобразование категориальных переменных в числовые методом get_dummies
X_train = pd.get_dummies(X_train, columns=['Embarked', 'Sex', 'Cabin'], drop_first=True)

In [9]:
# Создаем объект LabelEncoder
label_encoder = LabelEncoder()

# Преобразуем столбец 'Name'
X_train['Name'] = label_encoder.fit_transform(X_train['Name'])

# Преобразуем столбец 'Ticket'
X_train['Ticket'] = label_encoder.fit_transform(X_train['Ticket'])

# Инициализация объекта MinMaxScaler
scaler = MinMaxScaler()

# Выбор числовых признаков, которые нужно масштабировать
numeric_features = ['Age', 'Fare', 'Ticket', 'Name', 'Pclass', 'SibSp', 'Parch']

# Масштабирование выбранных признаков
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])

In [10]:
X_train.to_csv('data\\X_train_proceed.csv', index=False)